# OpenMPW bringup notebook

This goal of this notebook is to help you share a reproducible bringup methodology for your [OpenMPW](https://developers.google.com/silicon) project.

- Cells without `%` [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) run on the micropython board.
- Cells with the `%local` magic run on the host computer.

This assumes that the board has already been flashed using the [installation](https://github.com/efabless/caravel_board/tree/main/firmware_vex/nucleo#installation) instructions.

## Connect to the nucleo board

In [21]:
%local
devices=!mpremote connect list | cut -d ' ' -f 1
NUCLEO_DEVICE=devices[0]
NUCLEO_DEVICE

'/dev/ttyACM0'

In [23]:
%serialconnect /dev/ttyACM0
# update the device name to match the device detected


 ** Serial connected **



INFO:micropython-upydevice:Device pyboard connected in /dev/ttyACM0


SerialDevice @ /dev/ttyACM0, Type: pyboard, Class: SerialDevice
Firmware: MicroPython v1.19.1-616-g5987130af-dirty on 2022-11-20; NUCLEO-F746ZG with STM32F746
STM32 STLink, Manufacturer: STMicroelectronics
(MAC: 2f:00:4d:00:02:50:46:56:32:38:36:20)

MicroPython v1.19.1-616-g5987130af-dirty on 2022-11-20; NUCLEO-F746ZG with STM32F746
Type "help()" for more information.


## Run the diagnostic

In [38]:
# update those variable to match the part under test

SHUTTLE='MPW2'  # OpenMPW shuttle ID
SLOT='A5'       # project slot ID
DIE='0'         # actual die ID

PART_NAME=f'{SHUTTLE}_{SLOT}_{DIE}'
print('part name:', PART_NAME)


part name: MPW2_A5_0


In [52]:
import io_config

io_config.run(part_name=PART_NAME)


 
io_config -- version 1.2.0
voltage = 1.60, analog = False
 
== Beginning IO configuration test.  Testing LOW IO chain...     ==
 
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_INDEPENDENT >> Failed
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_DEPENDENT   >> Passed
gpio[03] - H_INDEPENDENT >> Passed
gpio[04] - H_INDEPENDENT >> Passed
gpio[05] - H_INDEPENDENT >> Passed
gpio[06] - H_INDEPENDENT >> Passed
gpio[07] - H_INDEPENDENT >> Passed
gpio[08] - H_INDEPENDENT >> Failed
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_DEPENDENT   >> Passed
gpio[03] - H_INDEPENDENT >> Passed
gpio[04] - H_INDEPENDENT >> Passed
gpio[05] - H_INDEPENDENT >> Passed
gpio[06] - H_INDEPENDENT >> Passed
gpio[07] - H_INDEPENDENT >> Passed
gpio[08] - H_DEPENDENT   >> Passed
gpio[09] - H_INDEPENDENT >> Passed
gpio[10] - H_INDEPENDENT >> Failed
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_DEP

In [66]:
f = open('gpio_config_def.py')
GPIO_CONFIG = f.read()
print(GPIO_CONFIG)
GPIO_CONFIG


# gpio_config_def.py file for part A5_0
# io_config -- version 1.2.0
voltage = 1.60
analog = False

H_NONE        = 0  
H_DEPENDENT   = 1  
H_INDEPENDENT = 2  
H_SPECIAL     = 3  
H_UNKNOWN     = 4  

# voltage: 1.6
# configuration failed in gpio[10], anything after is invalid
gpio_l = [
['IO[0]', H_NONE],
['IO[1]', H_INDEPENDENT],
['IO[2]', H_DEPENDENT],
['IO[3]', H_INDEPENDENT],
['IO[4]', H_INDEPENDENT],
['IO[5]', H_INDEPENDENT],
['IO[6]', H_INDEPENDENT],
['IO[7]', H_INDEPENDENT],
['IO[8]', H_DEPENDENT],
['IO[9]', H_INDEPENDENT],
['IO[10]', H_UNKNOWN],
['IO[11]', H_UNKNOWN],
['IO[12]', H_UNKNOWN],
['IO[13]', H_UNKNOWN],
['IO[14]', H_UNKNOWN],
['IO[15]', H_UNKNOWN],
['IO[16]', H_UNKNOWN],
['IO[17]', H_UNKNOWN],
['IO[18]', H_UNKNOWN],
]
# voltage: 1.6
# configuration failed in gpio[34], anything before is invalid
gpio_h = [
['IO[37]', H_NONE],
['IO[36]', H_DEPENDENT],
['IO[35]', H_INDEPENDENT],
['IO[34]', H_UNKNOWN],
['IO[33]', H_UNKNOWN],
['IO[32]', H_UNKNOWN],
['IO[31]', H_UNKNOWN],

## Run a sanity check

In [58]:
import io_config

io_config.run_sanity_check()


 
io_config -- version 1.2.0
voltage = 1.60, analog = False
== Beginning SANITY for LOW IO chain...                          ==
 
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_INDEPENDENT >> Passed
gpio[03] - H_INDEPENDENT >> Passed
gpio[04] - H_INDEPENDENT >> Passed
gpio[05] - H_INDEPENDENT >> Passed
gpio[06] - H_INDEPENDENT >> Passed
gpio[07] - H_INDEPENDENT >> Passed
gpio[08] - H_INDEPENDENT >> Passed
gpio[09] - H_INDEPENDENT >> Passed
gpio[10] - H_INDEPENDENT >> Failed
**** SANITY CHECK FOR LOW CHAIN PASSED!!
 
== LOW IO chain test complete.  Testing HIGH IO chain...         ==
 
::::: flashing Caravel :::::
gpio[36] - H_INDEPENDENT >> Passed
gpio[35] - H_INDEPENDENT >> Passed
gpio[34] - H_INDEPENDENT >> Passed
gpio[33] - H_INDEPENDENT >> Failed
**** SANITY CHECK FOR HIGH CHAIN FAILED!!
 
== HIGH IO chain test complete. SANITY test complete.            ==
 
 
== LOW chain PASSED.                                             ==
== HIGH chain FAILED.   

## Run the GPIO test

In [23]:
%local
!rshell -p {NUCLEO_DEVICE} cp /flash/gpio_config_def.py ../firmware_vex/gpio_test/
!make -C ../firmware_vex/gpio_test/ gpio_test.hex
!rshell -p {NUCLEO_DEVICE} cp ../firmware_vex/gpio_test/gpio_test.hex /flash/firmware.hex
!echo done

Welcome to rshell. Use Control-D (or the exit command) to exit rshell.

No MicroPython boards connected - use the connect command to add one

/home/proppy/src/github.com/efabless/caravel_board/notebooks> 

Using buffer-size of 32
Connecting to /dev/ttyACM0 (buffer-size 32)...
Trying to connect to REPL  connected
Retrieving sysname ... pyboard
Testing if ubinascii.unhexlify exists ... Y
Retrieving root directories ... /flash/
Setting time ... Feb 16, 2023 16:14:01
Evaluating board_name ... pyboard
Retrieving time epoch ... Jan 01, 2000
Copying '/flash/gpio_config_def.py' to '/home/proppy/src/github.com/efabless/caravel_board/firmware_vex/gpio_test/gpio_config_def.py' ...
make: Entering directory '/home/proppy/src/github.com/efabless/caravel_board/firmware_vex/gpio_test'
python3 ../gpio_config/gpio_config_builder.py
stream_h   = 11100000000011110000000001111000000000111100000000011110000000001111000000000111100000000011110000000001111000000000111100000000011110000000001111000000000111100

In [27]:
import io_config

io_config.run_flash_caravel()


*** flashing Caravel
status Good


## Run your own firmware

In [46]:
%local
%%writefile ../firmware_vex/gpio_test/gpio_config_io.py
# number of IO in the configuration stream for each chain
NUM_IO = 19

# defines these values for IO configurations
C_MGMT_OUT = 0
C_MGMT_IN = 1
C_USER_BIDIR = 2
C_DISABLE = 3
C_ALL_ONES = 4
C_USER_BIDIR_WPU = 5
C_USER_BIDIR_WPD = 6
C_USER_IN_NOPULL = 7
C_USER_OUT = 8

config_h = [
    C_MGMT_OUT,  #37
    C_MGMT_OUT,  #36
    C_MGMT_OUT,  #35
    C_DISABLE,  #34
    C_DISABLE,  #33
    C_DISABLE,  #32
    C_DISABLE,  #31
    C_DISABLE,  #30
    C_DISABLE,  #29
    C_DISABLE,  #28
    C_DISABLE,  #27
    C_DISABLE,  #26
    C_DISABLE,  #25
    C_DISABLE,  #24
    C_DISABLE,  #23
    C_DISABLE,  #22
    C_DISABLE,  #21
    C_DISABLE,  #20
    C_DISABLE,  #19
]

del config_h[NUM_IO:]

config_l = [
    C_DISABLE,   #0
    C_MGMT_OUT,   #1
    C_MGMT_OUT,   #2
    C_MGMT_OUT,   #3
    C_MGMT_OUT,   #4
    C_MGMT_OUT,   #5
    C_MGMT_OUT,   #6
    C_MGMT_OUT,   #7
    C_MGMT_OUT,   #8
    C_MGMT_OUT,   #9
    C_DISABLE,   #10
    C_DISABLE,   #11
    C_DISABLE,   #12
    C_DISABLE,   #13
    C_DISABLE,   #14
    C_DISABLE,   #15
    C_DISABLE,   #16
    C_DISABLE,   #17
    C_DISABLE,   #18
]

del config_l[NUM_IO:]

Overwriting ../firmware_vex/gpio_test/gpio_config_io.py


In [47]:
%local
%%writefile ../firmware_vex/gpio_test/gpio_test.c
#include "../defs.h"
#include "../gpio_config/gpio_config_io.c"

void set_registers() {
    reg_mprj_io_1 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_2 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_3 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_4 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_5 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_6 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_7 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_8 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_9 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_35 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_36 = GPIO_MODE_MGMT_STD_OUTPUT;
    reg_mprj_io_37 = GPIO_MODE_MGMT_STD_OUTPUT;
}

void main()
{
    reg_gpio_mode1 = 1;
    reg_gpio_mode0 = 0;
    reg_gpio_ien = 1;
    reg_gpio_oe = 1;

    set_registers();
    reg_mprj_datah = 0;
    reg_mprj_datal = 0;
    gpio_config_io();

    reg_gpio_out = 1; // OFF

    while(1) {
        // blink slowly

        reg_mprj_datal = 0x00000000;
        reg_mprj_datah = 0x00000000;

        reg_gpio_out = 0x0;

        delay(5000000);

        reg_mprj_datal = 0xffffffff;
        reg_mprj_datah = 0xffffffff;

        reg_gpio_out = 0x1;

        delay(5000000);

    }

}

Overwriting ../firmware_vex/gpio_test/gpio_test.c


In [48]:
%local
!make -C ../firmware_vex/gpio_test/ gpio_test.hex
!rshell -p {NUCLEO_DEVICE} cp ../firmware_vex/gpio_test/gpio_test.hex /flash/firmware.hex
!echo done

make: Entering directory '/home/proppy/src/github.com/efabless/caravel_board/firmware_vex/gpio_test'
python3 ../gpio_config/gpio_config_builder.py
stream_h   = 000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011100000000011100000000011100000000001
stream_l   = 000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111100000000111100000000111100000000011100000000011100000000011100000000011100000000011100000000011100000000000000000000000
n_bits = 247
xpack-riscv-none-elf-gcc-11.3.0-1/bin/riscv-none-elf-gcc -I/home/proppy/src/github.com/efabless/caravel_board/firmware_vex/gpio_test -I../ -I../generated/ -O0 -mabi=ilp32 -march=rv32i -D__vexriscv__ -Wl,-Bstatic,-T,../sections.lds,--strip-debug -ffreestanding -nostdlib -o gpio_test.elf ../crt0_vex.S ../isr.c gpio_test.

In [50]:
import io_config

io_config.run_flash_caravel()


*** flashing Caravel
status Good
